In [95]:
import pandas as pd
import sys
import os
from os import listdir
os.chdir("c:/Users/Robert/Documents/Projekte/dev/sport_betting/")
import config as CONFIG
pd.set_option('display.max_columns', 500)

from src.features import load_team_opponent
from src.features.preprocess import derived_odds,get_bookmaker, get_odd_pred, split_date,add_last_3_scores_column
import numpy as np

# Lade Daten ein

## Team Opponent

In [96]:
df = load_team_opponent(filename_main = CONFIG.DATA_FOLDER_RAW+'main_leagues_database_raw.sav')


C:\Users\Robert\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  """Entry point for launching an IPython kernel.


## Sortiere den Datensatz nach Datum, für die Durchschnittsfelder

In [97]:
df = split_date(df)
df = df.sort_values(['year','month','day'],ascending=True)
df = df.reset_index()

In [98]:
df["diff_FTG_against_Opponent"] = df["FTG_Team"]-df["FTG_Opponent"]
df["diff_ShotsOnTarget_against_Opponent"] = df["ShotsOnTarget_Team"]-df["ShotsOnTarget_Opponent"]

In [99]:
df = add_last_3_scores_column(df = df, score_column="diff_FTG_against_Opponent",anz_games=4)
df = add_last_3_scores_column(df = df, score_column="diff_ShotsOnTarget_against_Opponent",anz_games=4)
df = add_last_3_scores_column(df = df, score_column="B365_Team",anz_games=4)
df = add_last_3_scores_column(df = df, score_column="B365_Opponent",anz_games=4)
df = df.dropna()

# Erstelle Zielvariable

In [100]:
df['Target'] = df.FTG_Team>df.FTG_Opponent
df['Target'].value_counts()

False    2065
True     1259
Name: Target, dtype: int64

# Features entwicklen

In [101]:
standad_attributes = ['Target','atHome','Date','Div']
bookmaker_attributes = get_bookmaker(bm = 'BW',df = df)+get_bookmaker(bm='B365',df = df)+get_bookmaker(bm='IW',df = df)
bet_on_team = get_bookmaker(bm='Team',df=df[bookmaker_attributes])
bet_on_opponent = get_bookmaker(bm='Opponent',df=df[bookmaker_attributes])
bet_on_draw = get_bookmaker(bm='Draw',df=df[bookmaker_attributes])


In [102]:
df_derived_b365 = get_odd_pred(bet="B365",df=df)
df_derived_iw = get_odd_pred(bet="IW",df=df)
df_derived_bw = get_odd_pred(bet="BW",df=df)


In [103]:
df_derived_team = derived_odds(sight = 'Team',df=df, odds=bet_on_team)
df_derived_opponent = derived_odds(sight = 'Opponent',df=df, odds=bet_on_opponent)
df_derived_draw = derived_odds(sight = 'Draw',df=df, odds=bet_on_draw)


AttributeError: 'float' object has no attribute 'dtype'

# Modelldatensatz erstellen

In [ ]:
model_data = pd.concat([df[standad_attributes],df[bookmaker_attributes],df[["Team","month","year"]],df_derived_team,df_derived_opponent,df_derived_draw,df_derived_b365,df_derived_iw,df_derived_bw],axis=1)

In [ ]:
model_data = pd.concat([df[standad_attributes],df_derived_team,df_derived_opponent,df_derived_draw,df[bookmaker_attributes],df[["Div","Team"]]],axis=1)
#model_data = pd.concat([df[standad_attributes],df[bookmaker_attributes],df_derived_team,df_derived_opponent,df_derived_draw],axis=1)
model_data = pd.concat([df[standad_attributes],df[bookmaker_attributes],df[["Team","month","year"]],df_derived_team,df_derived_opponent,df_derived_draw,df_derived_b365,df_derived_iw,df_derived_bw],axis=1)


In [ ]:
model_data.replace([np.inf, -np.inf], np.nan, inplace=True)
model_data =  model_data.dropna(0)

In [ ]:
anew_df = add_last_3_scores_column(df = df, score_column="B365_Team")

In [ ]:
from sklearn import preprocessing
le_div = preprocessing.LabelEncoder()
le_team = preprocessing.LabelEncoder()

le_team.fit(list(set(model_data.Team)))
model_data.Team = le_team.transform(model_data.Team)



# array([2, 2, 1]...)
# >>> list(le.inverse_transform([2, 2, 1]))
# ['tokyo', 'tokyo', 'paris']

In [ ]:
le_team.fit(list(set(model_data.Team)))
model_data.Team = le_team.transform(model_data.Team)

In [ ]:
le_div.fit(list(set(model_data.Div)))
model_data.Div = le_div.transform(model_data.Div)


In [ ]:
# drop_list = ["Date","Team","Div",'BW_Opponent', 'BW_Draw', 'BW_Team',
#        'B365_Opponent', 'B365_Draw', 'B365_Team', 'IW_Opponent', 'IW_Draw',
#        'IW_Team', 'month', 'year', 'Max_Team', 'Min_Team', 'Avg_Team',
#        'Span_Team', 'Ratio_Team', 'Max_Opponent', 'Min_Opponent',
#        'Avg_Opponent', 'Span_Opponent', 'Ratio_Opponent', 'Max_Draw',
#        'Min_Draw', 'Avg_Draw', 'Span_Draw', 'Ratio_Draw']
# model_data = model_data.drop(columns=drop_list)

Aufteilen in Training, Test und Validierung

In [ ]:
import datetime
datum_1_1_2020 = datetime.datetime(2021, 6, 1)
anwendung = model_data[model_data.Date>datum_1_1_2020]
model_data = model_data[model_data.Date<datum_1_1_2020]


In [ ]:
model_data.columns
model_data.to_csv(GLOBALS.DATA_FOLDER + 'model_data.csv')
anwendung.to_csv(GLOBALS.DATA_FOLDER + 'val_data.csv')

# Modellieren

In [ ]:
[c for c in df.columns if 'last' in c ]

In [ ]:
model_data = df[['FTG_Team_last_3_games',
 'diff_FTG_last_3_games',
 'diff_FTG_against_Opponent_last_4_games',
 'diff_ShotsOnTarget_against_Opponent_last_4_games',
 'B365_Team_last_4_games',
 'B365_Opponent_last_4_games','B365_Team','Target']]

In [ ]:
model_data = model_data.dropna()

In [ ]:
from sklearn.model_selection import train_test_split

y = model_data['Target']
X = model_data.drop(['Target'], axis=1)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0, stratify=y)

In [ ]:


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV  
from xgboost import XGBClassifier, XGBRegressor
model = RandomForestClassifier(
      n_estimators=200, random_state=0, verbose=1, min_samples_leaf=5, min_samples_split=10)
#model = XGBClassifier(n_estimators=200,random_state = 0,verbose=1, min_samples_leaf= 5, min_samples_split = 10)
parameters = {
	'selector__threshold': [0],
	'classifier__n_neighbors': [1],
	'classifier__max_depth': [5],
	'classifier__learning_rate': [0.01],
	'classifier__n_estimators': [100,]
}
 
 
#model = GridSearchCV(XGBClassifier(), parameters,cv=10,scoring = 'precision').fit(X_train, y_train)
#model = gridsearch(model, X_train, y_train)
#model = XGBRegressor()
model.fit(X=X_train, y=y_train)


In [ ]:
[*zip(X.columns ,model.feature_importances_)]

In [ ]:
from sklearn.metrics import *

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
precision_score(model.predict(X_test),y_test)

In [ ]:
true = X_test[y_test!=y_pred]
df.loc[true.index][['Team','Opponent','FTG_Team','FTG_Opponent']]

In [ ]:
df.iloc[8773]

In [ ]:
y_anw = anwendung['Target']
X_anw  = anwendung.drop(['Target'], axis=1)

evaluate_model(model, y_anw, X_anw, show=False,threshold=0.5)

In [ ]:

evaluate_model(model, y_test, X_test, show=False,threshold=0.5)

In [ ]:
model_data.dtypes

In [ ]:
y_pred_proba = model.predict_proba(X_anw)[:,1]
y_pred_proba = pd.Series(y_pred_proba,index=X_anw.index,name='pred')
results = pd.concat([y_pred_proba,y_anw,df.FTG_Team,df.FTG_Opponent,df.BW_Team,df.BW_Opponent,df.BW_Draw,df.IW_Team,df.IW_Opponent,df.IW_Draw,df.B365_Team,df.B365_Opponent,df.B365_Draw,df.Date,df.year,df.month,df.day],axis=1,join='inner')
results

In [ ]:
stichtage =results[["year","month","day"]].drop_duplicates()
stichtage

In [ ]:
r = evaluate_my_bets(bet='BW',pred_min=pred_min,odd_min=odd_min,results = match_day,method='kombi')
r>0

In [ ]:
i=20
year = stichtage.year.iloc[i]
month = stichtage.month.iloc[i]
day =stichtage.day.iloc[i]
match_day = results[(results.year == year)&(results.month == month)&(results.day == day)]
ren = []
pred_min = 0.6
odd_min = 1.8
for i in range(0,len(stichtage)):
    year = stichtage.year.iloc[i]
    month = stichtage.month.iloc[i]
    day =stichtage.day.iloc[i]
    match_day = results[(results.year == year)&(results.month == month)&(results.day == day)]
    r = evaluate_my_bets(bet='BW',pred_min=pred_min,odd_min=odd_min,results = match_day,method='kombi')
    if r>0:
        print(i)
        ren +=[r]
        #ren +=[match_day.Target[(match_day.pred>pred_min)&(match_day.BW_Team>odd_min)]]

sum(ren)

In [ ]:
sum(ren)/len(stichtage)

In [ ]:
bet = 'BW'
pred_min = 0.6
odd_min = 2.1
def evaluate_my_bets(bet,pred_min,odd_min,results,method = 'single'):
    pred_bet = results[bet+"_Team"]/((results[bet+"_Team"]+results[bet+"_Opponent"]+results[bet+"_Draw"]))

    kandidaten = results[results.pred>pred_bet][results.pred>pred_min][results[bet+"_Team"]>odd_min]

    kandidaten = kandidaten[["Target","FTG_Team","FTG_Opponent",bet+"_Team","pred"]].join(df["Div"])

    
    if kandidaten.shape[0]>0:
        if method == 'single':
            rendite = sum(kandidaten[df.Target][bet+"_Team"])/kandidaten.shape[0]
        elif method == 'kombi':
            bet_on = (match_day.pred>pred_min)&((match_day[bet+"_Team"]>odd_min))
            bets = match_day.Target[bet_on]
            if np.prod(bets) == 1:
                rendite = np.prod(match_day[bet+"_Team"][bet_on])
            else: 
                rendite = 0
    else:
        rendite = 0
    # print("getippte Spiele",kandidaten.shape[0])
    # print("Richtige Spiele:",sum(kandidaten.Target))
    # print("Rendite:",rendite)
    kandidaten
    return rendite

evaluate_my_bets(bet='IW',pred_min=0.7,odd_min=1,results = match_day,method = 'kombi')

In [ ]:
            bet_on = (match_day.pred>0.4)&((match_day[bet+"_Team"]>odd_min))
            bets = match_day.Target[bet_on]
            bets

In [ ]:
match_day
bets = match_day.Target[match_day.pred>pred_min]
print(np.prod(bets))
if np.prod(bets) == 1:
    rendite = np.prod(match_day[bet+"_Team"][match_day.pred>pred_min])
rendite

#match_day[match_day.Target==(match_day.pred>0.5)][match_day.Target].BW_Team

In [ ]:
match_day

In [ ]:
evaluate_my_bets(bet='BW',pred_min=0.7,odd_min=2.0,results = results[results.year==2021])
evaluate_my_bets(bet='BW',pred_min=0.7,odd_min=2.,results = results[results.year==2022])

In [ ]:
umsatz  =[]
for p in range(50,100):
    p = p/100
    umsatz += [evaluate_my_bets(bet='IW',pred_min=p,odd_min=2,results = results[results.year==2019])]


max(umsatz)


In [ ]:
evaluate_my_bets(bet='IW',pred_min=0.6,odd_min=2,results = results[results.year==2019])